In [1]:
pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 7.0 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cesium 0.12.4 requires numpy<3.0,>=2.0, but you have numpy 1.26.4 which is incompatible.
bigframes 1.42.0 requires rich<14,>=12.4.4, but you have rich 14.0.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.9.0.13 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cudnn-cu12==9.1.0.70; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cudnn

In [2]:
import warnings
warnings.filterwarnings('ignore')

import os
import librosa
import random
import torch
import evaluate
import torchaudio
import numpy as np
import pandas as pd
from datasets import Dataset
from dataclasses import dataclass
from typing import Any, Dict, List, Union
from transformers import Seq2SeqTrainer
from transformers import Seq2SeqTrainingArguments
from transformers import WhisperTokenizer, WhisperProcessor, WhisperFeatureExtractor, WhisperForConditionalGeneration

2025-06-23 11:09:41.479299: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1750676981.678504      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1750676981.733842      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
dataset_root = "/kaggle/input/practice-dataset3/asr_dataset"
audio_dir = os.path.join(dataset_root, "audios")
csv_path = os.path.join(dataset_root, "transcriptions.csv")

In [4]:
df = pd.read_csv(csv_path)
df["path"] = df["audio_name"].apply(lambda x: os.path.join(audio_dir, x))
df.rename(columns={"sentence": "sentence"}, inplace=True)

In [5]:
df

,audio_name,sentence,path
0,p1_1.mp3,এই চিকেন নেক যদি কাট হয়ে যায় তাহলে আসলে সেভে...,/kaggle/input/practice-dataset3/asr_dataset/au...
1,p1_2.mp3,থ্রু বাংলাদেশ ফেনিতে একটা ব্রিজ উদ্বোধন করতে ম...,/kaggle/input/practice-dataset3/asr_dataset/au...
2,p1_3.mp3,থেকে মালামাল ত্রিপুরাতে পাঠানো যাবে ওরা যেটা ক...,/kaggle/input/practice-dataset3/asr_dataset/au...
3,p1_4.mp3,মাইন্ডসেটটা ঢুকায় দেওয়ার চেষ্টা করতেছে একমাত...,/kaggle/input/practice-dataset3/asr_dataset/au...
4,p1_5.mp3,ডিফেন্ডারস প্যারাডাইজ যুদ্ধ মানেই আমরা খুব কুই...,/kaggle/input/practice-dataset3/asr_dataset/au...
...,...,...,...
4377,q5_260.mp3,"স্বেচ্ছাশ্রমগুলো করেন, বিএনসিসি করেন, গার্লস গ...",/kaggle/input/practice-dataset3/asr_dataset/au...
4378,q5_261.mp3,জীবনে আর পাঁচজনের চাইতে অনেক বেশি এগিয়ে যান। ...,/kaggle/input/practice-dataset3/asr_dataset/au...
4379,q5_262.mp3,"50 ভাগ আমাদের এই স্বেচ্ছা কার্যক্রম, ক্লাব কার...",/kaggle/input/practice-dataset3/asr_dataset/au...
4380,q5_263.mp3,আমাদের সন্তানদেরকে আগামী দিনের জন্য তৈরি করি। ...,/kaggle/input/practice-dataset3/asr_dataset/au...


In [6]:
from sklearn.model_selection import train_test_split
from datasets import Dataset

# First split: 90% train_val, 10% test
train_val_df, test_df = train_test_split(df, test_size=0.1, random_state=42)

# Second split: from train_val, 88.9% train, 11.1% val (to maintain 80/10 split overall)
train_df, val_df = train_test_split(train_val_df, test_size=0.1111, random_state=42)

# Convert to HuggingFace Datasets
train_data = Dataset.from_pandas(train_df)
val_data = Dataset.from_pandas(val_df)
test_data = Dataset.from_pandas(test_df)


In [7]:
val_data

Dataset({
    features: ['audio_name', 'sentence', 'path', '__index_level_0__'],
    num_rows: 439
})

In [8]:
test_data

Dataset({
    features: ['audio_name', 'sentence', 'path', '__index_level_0__'],
    num_rows: 439
})

In [9]:
train_data

Dataset({
    features: ['audio_name', 'sentence', 'path', '__index_level_0__'],
    num_rows: 3504
})

In [10]:
model_type = "openai/whisper-small"
feature_extractor = WhisperFeatureExtractor.from_pretrained(model_type)
tokenizer = WhisperTokenizer.from_pretrained(model_type, language="Bengali", task="transcribe")
processor = WhisperProcessor.from_pretrained(model_type, language="Bengali", task="transcribe")
model = WhisperForConditionalGeneration.from_pretrained(model_type)
model.config.forced_decoder_ids = None
model.config.suppress_tokens = []

preprocessor_config.json:   0%|          | 0.00/185k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/283k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/836k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.48M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/494k [00:00<?, ?B/s]

...

added_tokens.json:   0%|          | 0.00/34.6k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.19k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.97k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/967M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/3.87k [00:00<?, ?B/s]

In [11]:
def prepare_dataset(batch):
    speech_array, sampling_rate = torchaudio.load(batch["path"], format="mp3")
    speech_array = speech_array[0].numpy()
    speech_array = librosa.resample(speech_array, orig_sr=sampling_rate, target_sr=16000)
    batch["input_features"] = feature_extractor(speech_array, sampling_rate=16000).input_features[0]
    batch["labels"] = tokenizer(batch["sentence"]).input_ids
    return batch

In [12]:
train_data = train_data.map(prepare_dataset)
val_data = val_data.map(prepare_dataset)
test_data=test_data.map(prepare_dataset)

Map:   0%|          | 0/3504 [00:00<?, ? examples/s]

Map:   0%|          | 0/439 [00:00<?, ? examples/s]

Map:   0%|          | 0/439 [00:00<?, ? examples/s]

In [13]:
# === Data collator ===
@dataclass
class DataCollatorSpeechSeq2SeqWithPadding:
    processor: Any

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        input_features = [{"input_features": feature["input_features"]} for feature in features]
        batch = self.processor.feature_extractor.pad(input_features, return_tensors="pt")

        label_features = [{"input_ids": feature["labels"]} for feature in features]
        labels_batch = self.processor.tokenizer.pad(label_features, return_tensors="pt")
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        if (labels[:, 0] == self.processor.tokenizer.bos_token_id).all().cpu().item():
            labels = labels[:, 1:]

        batch["labels"] = labels
        return batch

In [14]:
data_collator = DataCollatorSpeechSeq2SeqWithPadding(processor=processor)

In [15]:
pip install jiwer

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 19.9 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [16]:
# === Load both WER and CER metrics ===
wer_metric = evaluate.load("wer")
cer_metric = evaluate.load("cer")

def compute_metrics(pred):
    pred_ids = pred.predictions
    label_ids = pred.label_ids

    # Replace -100 in the labels as the tokenizer's pad_token_id
    label_ids[label_ids == -100] = tokenizer.pad_token_id

    # Decode predictions and references
    pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    label_str = tokenizer.batch_decode(label_ids, skip_special_tokens=True)

    # Compute WER and CER
    wer = 100 * wer_metric.compute(predictions=pred_str, references=label_str)
    cer = 100 * cer_metric.compute(predictions=pred_str, references=label_str)

    return {
        "wer": wer,
        "cer": cer,
    }


In [17]:
# # === Training Arguments ===
# training_args = Seq2SeqTrainingArguments(
#     # output_dir="./whisper-bn-finetuned",
#     per_device_train_batch_size=16,
#     per_device_eval_batch_size=16,
#     gradient_accumulation_steps=2,
#     learning_rate=1e-5,
#     warmup_steps=500,
#     num_train_epochs=8,
#     gradient_checkpointing=True,
#     fp16=True,
#     eval_strategy="epoch",
#     save_strategy='epoch',
#     save_steps=500,
#     eval_steps=100,
#     logging_steps=25,
#     predict_with_generate=True,
#     generation_max_length=225,
#     report_to=["tensorboard"],
#     load_best_model_at_end=True,
#     metric_for_best_model="wer",
#     greater_is_better=False,
#     push_to_hub=False,
# )
training_args = Seq2SeqTrainingArguments(
    output_dir="./whisper-bn-finetuned",
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    gradient_accumulation_steps=2,
    learning_rate=1e-5,
    warmup_steps=500,
    num_train_epochs=10,
    gradient_checkpointing=True,
    fp16=True,
    eval_strategy="epoch",
    save_strategy="no",
    logging_strategy="no",
    save_total_limit=1,
    save_steps=10000,
    eval_steps=100,
    logging_steps=50,
    predict_with_generate=True,
    generation_max_length=225,
    report_to=[],  # Disable tensorboard
    load_best_model_at_end=False,
    metric_for_best_model="wer",
    greater_is_better=False,
    push_to_hub=False,
)

In [18]:
from transformers import TrainerCallback
import pandas as pd

class EpochMetricsCallback(TrainerCallback):
    def __init__(self):
        self.epoch_data = []

    def on_log(self, args, state, control, logs=None, **kwargs):
        if logs and 'eval_loss' in logs:
            self.epoch_data.append({
                "epoch": round(state.epoch),
                "training_loss": logs.get("loss", None),
                "validation_loss": logs.get("eval_loss", None),
                "wer": logs.get("eval_wer", None),
                "cer": logs.get("eval_cer", None),  # If you're logging CER
            })

    def on_train_end(self, args, state, control, **kwargs):
        df = pd.DataFrame(self.epoch_data)
        df.to_csv("/kaggle/working/epoch_wise_metrics.csv", index=False)
        
    def on_evaluate(self, args, state, control, metrics=None, **kwargs):
        if metrics:
            self.epoch_data.append({
                "epoch": round(state.epoch) if state.epoch else "final_eval",
                "training_loss": None,
                "validation_loss": metrics.get("eval_loss", None),
                "wer": metrics.get("eval_wer", None),
                "cer": metrics.get("eval_cer", None),
            })


In [19]:
# === Trainer ===
trainer = Seq2SeqTrainer(
    args=training_args,
    model=model,
    train_dataset=train_data,
    eval_dataset=val_data,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=processor.feature_extractor,
    callbacks=[EpochMetricsCallback()],
)

In [20]:
# === Save processor and start training ===
# processor.save_pretrained(training_args.output_dir)
train_metrices = trainer.train()

Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.43.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.
`use_cache = True` is incompatible with gradient checkpointing. Setting `use_cache = False`...


Epoch,Training Loss,Validation Loss,Wer,Cer
1,No log,1.128394,108.521679,97.709389
2,No log,0.436531,76.176199,38.046389
3,No log,0.306224,61.427583,28.410168
4,No log,0.229756,55.154520,26.179294
5,No log,0.196652,49.838561,23.736250
6,No log,0.187682,47.013376,21.140774
7,No log,0.199494,46.090867,21.173732
8,No log,0.216203,46.171587,21.387962
9,No log,0.250777,46.655904,21.453879


Due to a bug fix in https://github.com/huggingface/transformers/pull/28687 transcription using a multilingual Whisper will default to language detection followed by transcription instead of translation to English.This might be a breaking change for your use case. If you want to instead always translate your audio to English, make sure to pass `language='en'`.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


In [21]:
train_metrices

TrainOutput(global_step=1090, training_loss=0.3194412371434203, metrics={'train_runtime': 19731.5387, 'train_samples_per_second': 1.776, 'train_steps_per_second': 0.055, 'total_flos': 1.002430246551552e+19, 'train_loss': 0.3194412371434203, 'epoch': 9.91324200913242})

In [22]:
import pandas as pd

# Convert metrics to DataFrame
df = pd.DataFrame([train_metrices.metrics])  # metrics is a dictionary

# Save to CSV (Kaggle outputs go in `/kaggle/working`)
df.to_csv('train_metrics.csv', index=False)

In [23]:
metrics = trainer.evaluate()
# print(metrics)  # This will include "eval_wer"

In [24]:
df = pd.DataFrame([metrics])
df.to_csv('eval_metrices.csv', index=False)

In [25]:
# Use trainer.predict if you want full batch evaluation
predictions = trainer.predict(test_data)
pred_ids = predictions.predictions

# Decode predictions to get actual transcriptions
decoded_preds = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)

In [26]:
len(decoded_preds)

439

In [27]:
import pandas as pd

# Create a list of dicts with your data
results = []

# Ensure range uses int, since len(val_data)/87 returns float
for i in range(0, int(len(test_data))):
    result = {
        "audio_name": test_data[i]["file_name"] if "file_name" in test_data[i] else os.path.basename(test_data[i]["path"]),
        "original": test_data[i]["sentence"],
        "predicted": decoded_preds[i]
    }
    results.append(result)

# Convert to DataFrame and save
df = pd.DataFrame(results)
df.to_csv("predictions.csv", index=False)


In [28]:
for i in range(0,len(test_data)):
    print("Audio Name : ", test_data[i]['audio_name'])
    print("Original  : ", test_data[i]['sentence'])
    print("Predicted : ", decoded_preds[i])
    print()

Audio Name :  p2_170.mp3
Original  :  কোভিড ব্লেসিং ইট মাইট সাউন্ড বাট পার্সোনালি ফর মাই ক্যারিয়ার আমার বাবার ক্যারিয়ারের জন্য ডিজাস্টার সে এক্সপোর্ট
Predicted :  হোজ এ ব্লেসিং ইট মাইট স্যাউন্ড হার্স বাট পার্স্নালি ফর মাই ক্যারিয়ার আমার বাবার ক্যারিয়ারেরের জন্য দ্যাট ওয়াজ 

Audio Name :  p4_402.mp3
Original  :  এর পয়েন্ট অফ ভিউতে কথা বলবো উপায় মানুষকে নিরুপায় করে উপায় নেওয়া হইছিল সরি উপায় মানুষকে অনেকটা নিরুপায় করে করছে কেন
Predicted :  এর কোয়্যান্টার বেথে কথা হবো উপায় মানুষ কি নিরুপাই করে উপায় নেওয়েছিল উপায় মানুষকে অনেকটা নিরুপাই করে করছে কেন

Audio Name :  p2_96.mp3
Original  :  নিশ্চে আরো 10 জনকে আপনাকে সাজেস্ট করবে ওইখানে নেটওয়ার্কটা ভাইয়া ক্রিয়েট করা ভেরি ইজি আর আপনার আপওয়ার্ক ফাইবার এ কি
Predicted :  নিশ্বে আর দোজন কে আপনি কায় সাড়েস্ট করবে ওইখানে নেটওয়ারকটা ভাই ক্রিয়েট করে ভেরি ইজি আর আপনার আপু ওয়ার্ক ফাইভ আর�

Audio Name :  p5_104.mp3
Original  :  আমি বলি আমাকে স্যার বইলেন আমাকে ভাই বলেন আমার ওই ব্যাপারটা কাজ করে আমাকে স্যার বলার দরকার নাই আমাকে ভাই বলেন তারপর আসলে
Predi